# Synthetic Strong Lens Generator

This notebook grabs pairs of spectra from observed DESI tiles and combines them according to the expression

$$
\mathcal{M}_3 = \alpha\mathcal{M}_1 + (1-\alpha)\mathcal{M}_2,
$$

where $\mathcal{M}_1$ is a model (redrock template) fit to the first observation and $\mathcal{M}_2$ is a model fit to the second spectrum. The value $\alpha$ is a uniformly generated parameter, e.g., $\alpha\sim U(0.1, 0.9)$. Given a model $\mathcal{M}_3$, the code computes a new realization of the flux of the combined spectra $f_3$.

Pairs of spectra are chosen (as of June 2021) from the same tile to ensure equal exposure times. Good redshift fits (`DELTACHI2>25` and `ZWARN==0`) are required. The member of the pair with the lower redshift value is referred to as the "lens," while the higher-redshift spectrum is the "background" object.

All spectra are saved in `desispec.Spectra` format. The following files are kept:
- `tileXXXXX_lens_spectra.fits`: Spectra of the lower-redshift objects in the selected pairs from tile ID XXXXX.
- `tileXXXXX_bkgd_spectra.fits`: Spectra of the higher-redshift objects in the selected pairs from tile ID XXXXX.
- `tileXXXXX_simlens_spectra.fits`: Combined spectra of the pairs from tile XXXXX.

The `Spectra` are stored with two additional tables:
1. `extra`: a table of model fits from redrock, keyed by spectrograph ('b', 'r', 'z').
2. `extra_catalog`: a table of `ZBEST` redshift fit and template fit coefficients from redrock.

### For the Impatient

The majority of this notebook contains functions that handle the spectrum bookkeeping. The code that actually generates the lenses is in the bottom 1/3 of the notebook starting in the section on [Generating Lenses](#generate_lens).

In [5]:
from glob import glob

from astropy.io import fits
from astropy.table import Table, join, vstack, hstack, unique

from desispec.io import read_spectra, write_spectra
from desispec.spectra import stack as specstack
from desispec.spectra import Spectra
from desispec.coaddition import coadd, coadd_cameras
from desispec.interpolation import resample_flux
from desispec.resolution import Resolution
from desispec.specscore import compute_coadd_scores

import redrock.templates

import copy

import os

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [6]:
mpl.rc('font', size=16)
mpl.rc('axes', titlesize='medium')

## Set up Redrock Templates

Instantiate a set of redrock templates for later use in constructing $\mathcal{M}_1$ and $\mathcal{M}_2$.

In [7]:
templates = dict()
for f in redrock.templates.find_templates():
    t = redrock.templates.Template(f)
    templates[(t.template_type, t.sub_type)] = t

DEBUG: Read templates from /global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/code/redrock-templates/master
DEBUG: Using default redshift range -0.0050-1.6997 for rrtemplate-galaxy.fits
DEBUG: Using default redshift range 0.0500-5.9934 for rrtemplate-qso.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-A.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-B.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-CV.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-F.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-G.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-K.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-M.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-WD.fits


## Bookkeeping Functions

A set of convenience functions to extract redshifts and spectra for the tile from a given spectroscopic reduction.

In [8]:
class SynthLens:
    
    def __init__(self, tileid, tilepath, tilefrac=0.15, alpha_min=0.1, alpha_max=0.9):
        """Generate synthetic strong lenses from one tile.
        
        Parameters
        ----------
        tileid : int
            DESI tile ID.
        tilepath : str
            Path to coadd and redrock FITS files for one tile.
        tilefrac : float
            Fraction of spectra in a tile to pair up in "lens" systems.
        alpha_min : float
            Number in [0,1] giving the minimum lens/background flux ratio.
        alpha_max : float
            Number in [0,1] giving the maximum lens/background flux ratio.
        """
        self.tileid   = tileid
        self.tilepath = tilepath
        self.tilefrac = tilefrac
        self.alphamin = alpha_min
        self.alphamax = alpha_max
        
        self.redshifts = None          # Table of redrock best-fit redshifts after quality cuts.
        self.fibermap = None           # Fibermap corresponding to extracted spectra.
        self.exp_fibermap = None       # Exposure fibermap corresponding to extra
        
        self.z_fgr = None              # Table of redshift data for the randomly selected low-redshift "foreground" object.
        self.z_bkg = None              # Table of redshift data for the randomly selected high-redshift "background" object.
        self.z_join = None             # A Cartesian join of zfgrtab and zbkgtab.
        self.z_sgl = None              # Table of redshift data for objects not used as synthetic strong lenses.
        
        self.target_fgr = None         # The TARGETID, TILEID, PETAL_LOC of the "foreground" objects in the lens system.
        self.target_bkg = None         # The TARGETID, TILEID, PETAL_LOC of the "background" objects in the lens system.
        self.target_sgl = None         # The TARGETID, TILEID, PETAL_LOC of objects *not* used as synthetic strong lenses.
        
        self.spec_sgl = None           # Spectra of single objects in the tile.
        self.spec_fgr = None           # Spectra of foreground objects (the lensing galaxies).
        self.spec_bkg = None           # Spectra of background objects.
        self.spec_len = None           # Spectra of synthetic strong lensing systems.
        
        # 1. Grab redrock data for this tile and extract good spectra.
        self._get_redrock_data()
        
        # 2. Randomly pair up objects to form "strong lenses."
        self._select_pairs()
        
        # 3. Extract coadd spectra for the selected objects.
        self._get_coadds()
        
        # 4. Add foreground and background spectra to produce lens spectra.
        self._generate_lens_spectra()
        
        # 5. Write output.
        self._write_fits()
        
    def _get_redrock_data(self):
        """Given a path to a tile folder, extract all REDSHIFT and EXP_FIBERMAP data from the tile.
            
        Returns
        -------
        nspec : int
            Number of spectra with passing redshifts pulled from this tile in total.
        """
        rrfiles = sorted(glob('{}/redrock*.fits'.format(self.tilepath)))
        
        # Loop through all files corresponding to individual petals.
        for rrfile in rrfiles:
            redshifts = Table.read(rrfile, 'REDSHIFTS')
            
            # Select all non-stellar objects with solid redshifts.
            select = (redshifts['SPECTYPE'] != 'STAR') & (redshifts['ZWARN'] == 0) & (redshifts['DELTACHI2'] >= 25) & (redshifts['TARGETID'] > 0)
            redshifts = redshifts[select]
            
            # Select the exp_fibermap table with all selected redshifts.
            exp_fmap = Table.read(rrfile, 'EXP_FIBERMAP')
            idx = np.in1d(exp_fmap['TARGETID'], redshifts['TARGETID'])
            exp_fmap = exp_fmap[idx]
            
            # Accumulate all redrock and exp_fibermap tables for the tile.
            if self.redshifts is None or self.exp_fibermap is None:
                self.redshifts = redshifts
                self.exp_fibermap = exp_fmap
            else:
                self.redshifts = vstack([self.redshifts, redshifts])
                self.exp_fibermap = vstack([self.exp_fibermap, exp_fmap])
        
        return len(self.redshifts)
    
    def _select_pairs(self):
        """Given a redshift and fibermap table, select random pairs of spectra (without replacement).

        Returns
        -------
        zlentab : astropy.Table
            Table of redshift data for the randomly selected low-redshift "lens."
        zbkgtab : astropy.Table
            Table of redshift data for the randomly selected high-redshift "background" object.
        zjointab : astropy.Table
            A Cartesian join of zlentab and zbkgtab for later use.
        znolentab : astropy.Table
            Table of redshift data for objects not used as synthetic strong lenses.
        lentargets : astropy.Table
            The TARGETID, TILEID, PETAL_LOC of the "lens" objects.
        bkgtargets : astropy.Table
            The TARGETID, TILEID, PETAL_LOC of the "background" objects.
        nolentargets : astropy.Table
            The TARGETID, TILEID, PETAL_LOC of objects not used as synthetic strong lenses.
        """
        # Select pairs of galaxies and QSOs.
        pairs = np.random.choice(self.redshifts['TARGETID'], [int(self.tilefrac*len(self.redshifts)), 2], replace=False)

        # Loop through pairs of TARGETIDs.
        for pair in pairs:
            # Check the fibermap table to ensure the exposure times of the lens and background are equal.
            # Unequal total exposure times could occur if a petal or a CANBus was disabled during one exposure. 
            select0 = np.in1d(self.exp_fibermap['TARGETID'], pair[0])
            select1 = np.in1d(self.exp_fibermap['TARGETID'], pair[1])
            i, j = np.where(select0)[0], np.where(select1)[0]
            exptime0, exptime1 = [np.sum(self.exp_fibermap[_]['EXPTIME']) for _ in (i,j)]
            if exptime0 != exptime1:
                continue

            # Sort to make index k the "lens" and index l the "background" galaxy.
            select = np.in1d(self.redshifts['TARGETID'], pair)
            pairdata = self.redshifts[select]
            k, l = np.where(select)[0] if pairdata[0]['Z'] < pairdata[1]['Z'] else np.where(select)[0][::-1]

            # Then join the two ZBEST table entries into one row.
            row = join(self.redshifts[k], self.redshifts[l], join_type='cartesian')

            # Generate a random number giving the relative contribution of objects 1 and 2.
            alpha = np.random.uniform(self.alphamin, self.alphamax)
            row['ALPHA'] = alpha

            # Accumulate the rows of pairs of "lenses" and "background" galaxies.
            if self.z_join is None:
                self.z_join = row
                self.z_fgr = self.redshifts[k]
                self.z_bkg = self.redshifts[l]
            else:
                self.z_join = vstack([self.z_join, row])
                self.z_fgr = vstack([self.z_fgr, self.redshifts[k]])
                self.z_bkg = vstack([self.z_bkg, self.redshifts[l]])

        # Index j selects rows from the fibermap that passed the redshift cuts.
        j = np.in1d(self.exp_fibermap['TARGETID'], self.redshifts['TARGETID'])

        # Index i selects targets chosen as "foreground" objects.
        # Remove these from index j.
        i = np.in1d(self.exp_fibermap['TARGETID'], self.z_fgr['TARGETID'])
        self.target_fgr = unique(self.exp_fibermap[i]['TARGETID', 'TILEID', 'PETAL_LOC'])

        j = ~i & j

        # Index i selects targets paired with lenses as "background" objects.
        # Remove these from index j.
        i = np.in1d(self.exp_fibermap['TARGETID'], self.z_bkg['TARGETID'])
        self.target_bkg = unique(self.exp_fibermap[i]['TARGETID', 'TILEID', 'PETAL_LOC'])

        j = ~i & j

        # Tabulate all objects with good redshifts that are neither "lenses" nor "background" objects.
        # Also output the ZBEST table for this list of objects.
        self.target_sgl = unique(self.exp_fibermap[j]['TARGETID', 'TILEID', 'PETAL_LOC'])
        idx = np.nonzero(self.target_sgl['TARGETID'][:,None] == self.redshifts['TARGETID'])[1]
        self.z_sgl = self.redshifts[idx]
        
    def _get_coadds(self):
        """Given REDSHIFT data and TARGET info, extract coadds to Spectra objects.
        """
        cofiles = sorted(glob('{}/coadd*.fits'.format(self.tilepath)))
        
        # Loop over all petals.
        ef_sgl, ef_fgr, ef_bkg = None, None, None
        
        for cofile in cofiles:
            petal, tile = [int(_) for _ in os.path.basename(cofile).split('-')[1:3]]
            coadds = read_spectra(cofile)
            
            # Extract spectra for the single-object (unpaired) galaxies in this petal.
            targetids = self.target_sgl[self.target_sgl['PETAL_LOC'] == petal]['TARGETID']
            i = np.in1d(coadds.fibermap['TARGETID'], targetids)
            j = np.in1d(coadds.exp_fibermap['TARGETID'], targetids)

            if self.spec_sgl is None:
                self.spec_sgl = coadds[i]
                ef_sgl = coadds.exp_fibermap[j]
            else:
                self.spec_sgl = specstack([self.spec_sgl, coadds[i]])
                ef_sgl = vstack([ef_sgl, coadds.exp_fibermap[j]])
            
            if coadds[i].exp_fibermap:
                print(len(coadds.exp_fibermap[i]))
                
            # Extract spectra for the foreground galaxies in this petal.
            targetids = self.target_fgr[self.target_fgr['PETAL_LOC'] == petal]['TARGETID']
            i = np.in1d(coadds.fibermap['TARGETID'], targetids)
            j = np.in1d(coadds.exp_fibermap['TARGETID'], targetids)

            if self.spec_fgr is None:
                self.spec_fgr = coadds[i]
                ef_fgr = coadds.exp_fibermap[j]
            else:
                self.spec_fgr = specstack([self.spec_fgr, coadds[i]])
                ef_fgr = vstack([ef_fgr, coadds.exp_fibermap[j]])
                
            # Extract spectra for the background galaxies in this petal.
            targetids = self.target_bkg[self.target_bkg['PETAL_LOC'] == petal]['TARGETID']
            i = np.in1d(coadds.fibermap['TARGETID'], targetids)
            j = np.in1d(coadds.exp_fibermap['TARGETID'], targetids)

            if self.spec_bkg is None:
                self.spec_bkg = coadds[i]
                ef_bkg = coadds.exp_fibermap[j]
            else:
                self.spec_bkg = specstack([self.spec_bkg, coadds[i]])
                ef_bkg = vstack([ef_bkg, coadds.exp_fibermap[j]])

        # Unscramble the indices so the order of the spectra matches our coadd.
        # Then store redrock outputs in the extra_catalog and extra members of Spectra.
        idx = np.nonzero(self.z_sgl['TARGETID'][:,None] == self.spec_sgl.fibermap['TARGETID'])[1]
        self.spec_sgl = self.spec_sgl[idx]
        self.spec_sgl.extra_catalog = self.z_sgl
        self.spec_sgl.extra = self._get_redrock_models(self.spec_sgl)
        self.spec_sgl.exp_fibermap = ef_sgl
        
        idx = np.nonzero(self.z_fgr['TARGETID'][:,None] == self.spec_fgr.fibermap['TARGETID'])[1]
        self.spec_fgr = self.spec_fgr[idx]
        self.spec_fgr.extra_catalog = self.z_fgr
        self.spec_fgr.extra = self._get_redrock_models(self.spec_fgr)
        self.spec_fgr.exp_fibermap = ef_fgr
        
        idx = np.nonzero(self.z_bkg['TARGETID'][:,None] == self.spec_bkg.fibermap['TARGETID'])[1]
        self.spec_bkg = self.spec_bkg[idx]
        self.spec_bkg.extra_catalog = self.z_bkg
        self.spec_bkg.extra = self._get_redrock_models(self.spec_bkg)
        self.spec_bkg.exp_fibermap = ef_bkg
        
    def _get_redrock_models(self, targspec):
        """Given Spectra + redshift, compute best-fit redrock model fluxes.
        
        Parameters
        ----------
        targspec : Spectra
            Input spectra with redshift info in the extra_catalog member.
        
        Returns
        -------
        model : dict
            Model fluxes keyed by spectrograph camera.
        """
        
        model = {}
        for band in 'brz':
            bandmodel = []
            
            for i in range(targspec.num_spectra()):
                z = targspec.extra_catalog[i]['Z']
                sp, sb = targspec.extra_catalog[i]['SPECTYPE'], targspec.extra_catalog[i]['SUBTYPE']
                ncoeff = templates[(sp, sb)].flux.shape[0]
                coeff = targspec.extra_catalog[i]['COEFF'][0:ncoeff]
                tflux = templates[(sp, sb)].flux.T.dot(coeff)
                twave = templates[(sp, sb)].wave * (1 + z)

                R = Resolution(targspec.resolution_data[band][i])
                txflux = R.dot(resample_flux(targspec.wave[band], twave, tflux))
                bandmodel.append(txflux)

            model[band] = { 'model' : np.asarray(bandmodel) }
        return model

#         # Turn off scores.
#         if not hasattr(targspec, 'scores_comments'):
#             targspec.scores_comments = None

#         return targspec

    def _generate_lens_spectra(self):
        """Generate realizations of strong lenses by adding the spectra.
        The result is combo = alpha*lens + (1-alpha)*bkgd
        """
        # Build up a list of arrays and dictionaries needed to instantiate Spectra.
        bands = []
        wave = {} 
        flux = {}
        ivar = {}
        mask = {}
        resolution = {}
        fibermap = None
        extra = {}
        extra_catalog=None

        # Loop through the observed bands and merge the model fits.
        for band in 'brz':
            f1, w1 = self.spec_fgr.flux[band], self.spec_fgr.ivar[band]
            m1 = []
            f2, w2 = self.spec_bkg.flux[band], self.spec_bkg.ivar[band]
            m2 = []
            alpha = self.z_join['ALPHA'][:,None]
            w3 = w1*w2 / (alpha*w2 + (1-alpha)*w1)

            # Add the models using the alpha parameter to tune the relative contribution of the lens and background object.
            m1 = self.spec_fgr.extra[band]['model']
            m2 = self.spec_bkg.extra[band]['model']
            m3 = alpha*m1 + (1-alpha)*m2

            # Compute a "noise" vector using the differences between the observed fluxes and model fits.
            n3 = np.sqrt(alpha*(f1-m1)**2 + (1-alpha)*(f2-m2)**2)

            # Create a realized flux as a Gaussian with expectation given by the model and 
            # width given by the noise vector.
            f3 = np.random.normal(loc=m3, scale=n3)

            # Set up the spectrum wavelength, flux, variance, mask, bands.
            wave[band] = self.spec_fgr.wave[band]
            flux[band] = f3
            ivar[band] = w3
            mask[band] = self.spec_fgr.mask[band] | self.spec_bkg.mask[band]
            resolution[band] = self.spec_fgr.resolution_data[band]       # Maybe try to add the resolution matrices properly?
            bands.append(band)
            extra[band] = { 'model' : m3 }

        # Set up the fibermap as a join of the lens and background fibermaps.
    #     for row1, row2 in zip(lenspec.fibermap, bkgspec.fibermap):
    #         newrow = join(row1, row2, join_type='cartesian')
    #         if fibermap is None:
    #             fibermap = newrow
    #         else:
    #             fibermap = vstack([fibermap, newrow])
    #     fibermap['TARGETID'] = fibermap['TARGETID_1']

        # Add redshift info from the two individual spectra as an extra catalog.
        extra_catalog = self.z_join

        self.spec_len = Spectra(bands, wave, flux, ivar, mask, resolution_data=resolution,
                                fibermap=self.spec_fgr.fibermap,
                                exp_fibermap=self.spec_fgr.exp_fibermap,
                                extra=extra,
                                extra_catalog=extra_catalog)
        
        compute_coadd_scores(self.spec_len)

    def _write_fits(self):
        """Write extracted spectra to FITS output.
        """
        write_spectra('tile{:06d}_sgl_spectra.fits'.format(self.tileid), self.spec_sgl)
        write_spectra('tile{:06d}_fgr_spectra.fits'.format(self.tileid), self.spec_fgr)
        write_spectra('tile{:06d}_bkg_spectra.fits'.format(self.tileid), self.spec_bkg)
        write_spectra('tile{:06d}_simlens_spectra.fits'.format(self.tileid), self.spec_len)
        
        # Coadd the cameras and store the output.
        self.spec_sgl.extra = None
        coadd = coadd_cameras(self.spec_sgl)
        write_spectra('tile{:06d}_sgl_coadd.fits'.format(self.tileid), coadd)
        
        self.spec_len.extra = None
        coadd = coadd_cameras(self.spec_len)
        write_spectra('tile{:06d}_simlens_coadd.fits'.format(self.tileid), coadd)

In [9]:
synthlens = SynthLens(1, '/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/1/20210406')

INFO:spectra.py:282:read_spectra: iotime 0.977 sec to read coadd-0-1-thru20210406.fits at 2021-08-02T18:38:32.395766
INFO:spectra.py:282:read_spectra: iotime 0.931 sec to read coadd-1-1-thru20210406.fits at 2021-08-02T18:38:35.130092
INFO:spectra.py:282:read_spectra: iotime 0.861 sec to read coadd-2-1-thru20210406.fits at 2021-08-02T18:38:38.309034
INFO:spectra.py:282:read_spectra: iotime 0.838 sec to read coadd-3-1-thru20210406.fits at 2021-08-02T18:38:41.962272
INFO:spectra.py:282:read_spectra: iotime 0.580 sec to read coadd-4-1-thru20210406.fits at 2021-08-02T18:38:46.175774
INFO:spectra.py:282:read_spectra: iotime 0.635 sec to read coadd-5-1-thru20210406.fits at 2021-08-02T18:38:50.904986
INFO:spectra.py:282:read_spectra: iotime 0.707 sec to read coadd-6-1-thru20210406.fits at 2021-08-02T18:38:55.550957
INFO:spectra.py:282:read_spectra: iotime 0.726 sec to read coadd-8-1-thru20210406.fits at 2021-08-02T18:39:01.791266
INFO:spectra.py:282:read_spectra: iotime 0.502 sec to read coadd

In [10]:
synthlens.spec_len.scores

{'TARGETID': <Column name='TARGETID' dtype='int64' length=424>
 39627805364848731
 39627817423475380
 39627871798427843
 39627877817254406
 39627841616216243
 39627823480049846
 39627859727221829
 39627823459078407
 39627877808866590
 39627823471662459
 39627859735610232
 39627835551256707
               ...
 39627829498872828
 39627847626656046
 39627811400454374
 39627847635044499
 39627853691621446
 39627841616218367
 39627859739804860
 39627823492633672
 39627859731416688
 39627865758632445
 39627853683234106
 39627829540818244,
 'INTEG_COADD_FLUX_B': array([ 1.58137903e+03,  4.63654663e+02,  1.19603235e+03,  1.36213562e+03,
         1.97823303e+03,  4.07619312e+03,  1.57910352e+03,  6.08987427e+02,
         1.78504651e+03,  4.28767822e+02,  1.05152673e+03,  2.37003735e+03,
         2.56852875e+02,  9.61154175e+02,  3.03968170e+02,  2.58943237e+02,
         1.52030289e+02,  2.15381201e+03,  4.08155975e+02,  5.50452026e+02,
         3.89368225e+02,  3.47725403e+02,  3.90010376e+02, 

In [37]:
synthlens.zsgltab

TARGETID,CHI2,COEFF [10],Z,ZERR,ZWARN,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2
int64,float64,float64,float64,float64,int64,int64,bytes6,bytes20,int64,float64
39627805356458119,9968.149694681168,0.0004030411999174501 .. 0.0,1.3877280585412985,0.00013538322659155844,0,7926,QSO,,4,990.9918611736502
39627805356458250,9427.643644802272,-8.382594832389221e-05 .. 0.0,1.025430405875676,0.0001741772994061619,0,7925,QSO,,4,446.6447897925973
39627805356458309,10448.540929943323,-78.48989653463316 .. -6.337248567649397,1.1235691113027995,8.124608402770733e-05,0,7925,GALAXY,,10,103.67351548373699
39627805356458488,9097.41621055454,7.658573954079481e-05 .. 0.0,1.8120840245409835,0.0004375385561060892,0,7926,QSO,,4,45.16224206984043
39627805356458530,9481.77835714817,-24.25138521140158 .. -0.266248728431096,1.431563323399275,5.1437348159985836e-05,0,7926,GALAXY,,10,47.08363455533981
39627805360652963,9936.057492733002,29.465846340682646 .. 1.702812586959047,0.8171459035925251,1.5430992984480902e-05,0,7925,GALAXY,,10,716.3360352516174
39627805360652990,8917.145603299141,54.365763598832736 .. -4.747169223189236,1.4243743162493747,5.027180884463072e-05,0,7925,GALAXY,,10,144.40822982788086
39627805360653613,8318.376172244549,75.32204747965794 .. 6.980505327277202,1.0723606007772009,4.677713254271739e-05,0,7924,GALAXY,,10,186.9498457312584
39627805360654247,8649.8765360713,84.73844539649566 .. -0.2640162928652584,0.8755976078627651,8.228468737756477e-05,0,7928,GALAXY,,10,161.63759247213602


In [ ]:
def get_zbest_data(tilefolder):
    """Given a path to a tile folder, extract all ZBEST and FIBERMAP data from the tile.
    
    Parameters
    ----------
    tilefolder : str
        Path to the spectroscopic reduction corresponding to a particular tile.
    
    Returns
    -------
    zbest : astropy.Table
        Table of best fit redshifts.
    fibermap : astropy.Table
        Fibermap table from this tile.
    """
    # List all zbest files in the tile folder.
    zbfiles = sorted(glob('{}/zbest*fits'.format(tilefolder)))
    zbest, fibermap = None, None
    
    # Spin through all files, corresponding to individual petals.
    for zbfile in zbfiles:
        _zbest = Table.read(zbfile, 'ZBEST')
        # Select galaxies and QSOs with a solid redshift.
        select = (_zbest['SPECTYPE'] != 'STAR') & (_zbest['ZWARN']==0) & (_zbest['DELTACHI2'] >=25 ) & (_zbest['TARGETID'] > 0)
        _zbest = _zbest[select]
        
        _fibermap = Table.read(zbfile, 'FIBERMAP')
        idx = np.in1d(_fibermap['TARGETID'], _zbest['TARGETID'])
        n = len(_zbest)
        _fibermap = _fibermap[idx]#[-n:]
        
        if zbest is None or fibermap is None:
            zbest = _zbest
            fibermap = _fibermap
        else:
            zbest = vstack([zbest, _zbest])
            fibermap = vstack([fibermap, _fibermap])
            
    return zbest, fibermap

In [ ]:
def select_pairs(zbest, fibermap, fraction=0.05):
    """Given a zbest and fibermap table, select random pairs of spectra (without replacement).
    
    Parameters
    ----------
    zbest : astropy.Table
        Table of best fit redshifts from a single tile.
    fibermap : astropy.Table
        Fibermap table from this tile.
    fraction : float
        Fraction of targets in the tile to use when generating pairs. Should be in [0,1].
        
    Returns
    -------
    zlentab : astropy.Table
        Table of redshift data for the randomly selected low-redshift "lens."
    zbkgtab : astropy.Table
        Table of redshift data for the randomly selected high-redshift "background" object.
    zjointab : astropy.Table
        A Cartesian join of zlentab and zbkgtab for later use.
    znolentab : astropy.Table
        Table of redshift data for objects not used as synthetic strong lenses.
    lentargets : astropy.Table
        The TARGETID, TILEID, PETAL_LOC of the "lens" objects.
    bkgtargets : astropy.Table
        The TARGETID, TILEID, PETAL_LOC of the "background" objects.
    nolentargets : astropy.Table
        The TARGETID, TILEID, PETAL_LOC of objects not used as synthetic strong lenses.
    """
    # Select pairs of galaxies and QSOs.
    pairs = np.random.choice(zbest['TARGETID'], [int(fraction*len(zbest)), 2], replace=False)

    # Loop through pairs of TARGETIDs.
    zlentab, zbkgtab, zjointab = None, None, None

    for pair in pairs:
        # Check the fibermap table to ensure the exposure times of the lens and background are equal.
        # Unequal total exposure times could occur if a petal or a CANBus was disabled during one exposure. 
        select0 = np.in1d(fibermap['TARGETID'], pair[0])
        select1 = np.in1d(fibermap['TARGETID'], pair[1])
        i, j = np.where(select0)[0], np.where(select1)[0]
        exptime0, exptime1 = [np.sum(fibermap[_]['EXPTIME']) for _ in (i,j)]
        if exptime0 != exptime1:
            continue

        # Sort to make index k the "lens" and index l the "background" galaxy.
        select = np.in1d(zbest['TARGETID'], pair)
        pairdata = zbest[select]
        k, l = np.where(select)[0] if pairdata[0]['Z'] < pairdata[1]['Z'] else np.where(select)[0][::-1]

        # Then join the two ZBEST table entries into one row.
        row = join(zbest[k], zbest[l], join_type='cartesian')

        # Generate a random number giving the relative contribution of objects 1 and 2.
        alpha = np.random.uniform(0.1, 0.9)
        row['ALPHA'] = alpha

        # Accumulate the rows of pairs of "lenses" and "background" galaxies.
        if zjointab is None:
            zjointab = row
            zlentab = zbest[k]
            zbkgtab = zbest[l]
        else:
            zjointab = vstack([zjointab, row])
            zlentab = vstack([zlentab, zbest[k]])
            zbkgtab = vstack([zbkgtab, zbest[l]])
    
    # Index j selects rows from the fibermap that passed the redshift cuts.
    j = np.in1d(fibermap['TARGETID'], zbest['TARGETID'])
    
    # Index i selects targets chosen as "lens" objects.
    # Remove these from index j.
    i = np.in1d(fibermap['TARGETID'], zlentab['TARGETID'])
    lentargets = unique(fibermap[i]['TARGETID', 'TILEID', 'PETAL_LOC'])
    
    j = ~i & j

    # Index i selects targets paired with lenses as "background" objects.
    # Remove these from index j.
    i = np.in1d(fibermap['TARGETID'], zbkgtab['TARGETID'])
    bkgtargets = unique(fibermap[i]['TARGETID', 'TILEID', 'PETAL_LOC'])
    
    j = ~i & j
    
    # Tabulate all objects with good redshifts that are neither "lenses" nor "background" objects.
    # Also output the ZBEST table for this list of objects.
    nolentargets = unique(fibermap[j]['TARGETID', 'TILEID', 'PETAL_LOC'])
    idx = np.nonzero(nolentargets['TARGETID'][:,None] == zbest['TARGETID'])[1]
    znolentab = zbest[idx]
    
    return zlentab, zbkgtab, zjointab, znolentab, lentargets, bkgtargets, nolentargets

In [ ]:
def get_coadds(ztab, targtab):
    """Given a table of ZBEST data and TARGET info, extract coadds to a Spectra object.
    
    Parameters
    ----------
    ztab : astropy.Table
        Table of ZBEST data of selected spectra.
    targab : astropy.Table
        The TARGETID, TILEID, PETAL_LOC of the selected spectra.
        
    Returns
    -------
    targspec : desispec.Spectra
        Spectra object with a list of spectra, including fibermaps, redshift fits (in extra_catalog), and template fits (in extra).
    """
    tile = targtab['TILEID'][0]
    tilefolder = sorted(glob('{}/{}/{}/*'.format(os.environ['DESI_SPECTRO_REDUX'], redux, tile)))[-1]

    targspec = None

    for petal in np.unique(targtab['PETAL_LOC']):
        coadd_file = glob('{}/coadd-{}*.fits'.format(tilefolder, petal))[0]
        coadds = read_spectra(coadd_file)

        # Extract selected spectra from this petal.
        targetids = targtab[targtab['PETAL_LOC'] == petal]['TARGETID']
        i = np.in1d(coadds.fibermap['TARGETID'], targetids)
        coadds = coadds[i]

        if targspec is None:
            targspec = coadds
        else:
            targspec = specstack([targspec, coadds])

    # Unscramble the indices so the order of the spectra matches our coadd...
    idx = np.nonzero(ztab['TARGETID'][:,None] == targspec.fibermap['TARGETID'])[1]
    targspec = targspec[idx]
    targspec.extra_catalog = ztab

    # Compute and store model fluxes from redrock.
    targspec.extra = {}
    for band in 'brz':
        model = []

        for i in range(targspec.num_spectra()):
            # Get the redrock best fit model for the lens.
            z = targspec.extra_catalog[i]['Z']
            sp, sb = targspec.extra_catalog[i]['SPECTYPE'], targspec.extra_catalog[i]['SUBTYPE']
            ncoeff = templates[(sp, sb)].flux.shape[0]
            coeff = targspec.extra_catalog[i]['COEFF'][0:ncoeff]
            tflux = templates[(sp, sb)].flux.T.dot(coeff)
            twave = templates[(sp, sb)].wave * (1 + z)

            R = Resolution(targspec.resolution_data[band][i])
            txflux = R.dot(resample_flux(targspec.wave[band], twave, tflux))
            model.append(txflux)

        targspec.extra[band] = { 'model' : np.asarray(model) }

    # Turn off scores.
    if not hasattr(targspec, 'scores_comments'):
        targspec.scores_comments = None
    
    return targspec

In [ ]:
def generate_lens_spectra(zjointab, lenspec, bkgspec, alpha_min=0.1, alpha_max=0.9):
    """Generate realizations of strong lenses by adding the spectra.
    The result is combo = alpha*lens + (1-alpha)*bkgd
    
    Parameters
    ----------
    zjointab : astropy.Table
        Joined table of redshift info from the lens and background objects.
    lenspec : desispec.Spectra
        Spectra for objects that are the "lenses" in our strong lens systems.
    bkgspec : desispec.Spectra
        Spectra for objects that are "backgrounds" in our strong lens systems.
    alpha_min : float
        Minimum alpha. Should be in [0,1] and < alpha_max.
    alpha_max : float
        Maximum alpha. Should be in [0,1] and > alpha_min.
        
    Returns
    -------
    newspec : Spectra
        A Spectra object with a list of synthetic lensed spectra.
    """
    # Build up a list of arrays and dictionaries needed to instantiate Spectra.
    bands = []
    wave = {} 
    flux = {}
    ivar = {}
    mask = {}
    resolution = {}
    fibermap = None
    extra = {}
    extra_catalog=None

    # Loop through the observed bands and merge the model fits.
    for band in 'brz':
        f1, w1 = lenspec.flux[band], lenspec.ivar[band]
        m1 = []
        f2, w2 = bkgspec.flux[band], bkgspec.ivar[band]
        m2 = []
        alpha = zjointab['ALPHA'][:,None]
        w3 = w1*w2 / (alpha*w2 + (1-alpha)*w1)

        # Add the models using the alpha parameter to tune the relative contribution of the lens and background object.
        m1 = lenspec.extra[band]['model']
        m2 = bkgspec.extra[band]['model']
        m3 = alpha*m1 + (1-alpha)*m2

        # Compute a "noise" vector using the differences between the observed fluxes and model fits.
        n3 = np.sqrt(alpha*(f1-m1)**2 + (1-alpha)*(f2-m2)**2)

        # Create a realized flux as a Gaussian with expectation given by the model and 
        # width given by the noise vector.
        f3 = np.random.normal(loc=m3, scale=n3)

        # Set up the spectrum wavelength, flux, variance, mask, bands.
        wave[band] = lenspec.wave[band]
        flux[band] = f3
        ivar[band] = w3
        mask[band] = lenspec.mask[band] | bkgspec.mask[band]
        resolution[band] = lenspec.resolution_data[band]       # Maybe try to add the resolution matrices properly?
        bands.append(band)
        extra[band] = { 'model' : m3 }

    # Set up the fibermap as a join of the lens and background fibermaps.
#     for row1, row2 in zip(lenspec.fibermap, bkgspec.fibermap):
#         newrow = join(row1, row2, join_type='cartesian')
#         if fibermap is None:
#             fibermap = newrow
#         else:
#             fibermap = vstack([fibermap, newrow])
#     fibermap['TARGETID'] = fibermap['TARGETID_1']

    # Add redshift info from the two individual spectra as an extra catalog.
    extra_catalog = zjointab

    newspec = Spectra(bands, wave, flux, ivar, mask, resolution_data=resolution,
                      fibermap=lenspec.fibermap,
                      extra=extra,
                      extra_catalog=extra_catalog)
    return newspec

## Generate Lensed Spectra
<a id='generate_lens'></a>

Build a list of SV tiles and generate synthetic strong lenses by pairing spectra.

In [ ]:
# Use the daily cumulative reductions, always grabbing the most recent spectra.
redux = 'daily/tiles/cumulative'

tiles = sorted(glob('{}/{}/*'.format(os.environ['DESI_SPECTRO_REDUX'], redux)))

sv1tiles = []
sv3tiles = []
for tile in tiles:
    tileid = int(os.path.basename(tile))
    d = sorted(glob('{}/*'.format(tile)))
    if tileid < 20000:
        sv3tiles.append(d[-1])
    if tileid > 80000:
        sv1tiles.append(d[-1])

In [ ]:
len(sv3tiles)

In [ ]:
nlens, nsgl = 0, 0

for i, sv3tile in enumerate(sv3tiles):
    # Extract ZBEST and FIBERMAP from all petals in this tile.
    zbest, fibermap = get_zbest_data(sv3tile)
    tileid = fibermap['TILEID'][0]
    print('\nGenerating strong lenses using TILE {}'.format(tileid), flush=True)
    
    # Extract redshift and target ID for randomly selected pairs of objects.
    # The "lens" data will be matched to the "bkg" data and will always have a lower redshift.
    zlentab, zbkgtab, zjointab, znolentab, lentargets, bkgtargets, nolentargets = select_pairs(zbest, fibermap, fraction=0.15)
    
    # Extract spectra + redrock fits to Spectra for objects not used to make synthetic strong lenses.
    sglspec = get_coadds(znolentab, nolentargets)    
    
    # Extract spectra + redrock fits to Spectra for the "lens" and "background" objects.
    lenspec = get_coadds(zlentab, lentargets)
    bkgspec = get_coadds(zbkgtab, bkgtargets)
    
    # Produce synthetic strong lenses.
    newspec = generate_lens_spectra(zjointab, lenspec, bkgspec)

    # Write outputs to FITS.
    write_spectra('tile{:06d}_singleobj_spectra.fits'.format(tileid), sglspec)
    write_spectra('tile{:06d}_lens_spectra.fits'.format(tileid), lenspec)
    write_spectra('tile{:06d}_bkgd_spectra.fits'.format(tileid), bkgspec)
    write_spectra('tile{:06d}_simlens_spectra.fits'.format(tileid), newspec)
    
    # Coadd the cameras of the single-object and simulated lens spectra.
    sglspec.extra = None
    sglspec_coadd = coadd_cameras(sglspec)
    write_spectra('tile{:06d}_singleobj_coadd.fits'.format(tileid), sglspec_coadd)
    
    newspec.extra = None
    newspec_coadd = coadd_cameras(newspec)
    write_spectra('tile{:06d}_simlens_coadd.fits'.format(tileid), newspec_coadd)
    
    # Check the cumulative statistics.
    nlens += newspec.num_spectra()
    nsgl += sglspec.num_spectra()
    print('Cumulative: {} single object spectra and {} "lens" spectra.'.format(nsgl, nlens), flush=True)
    
    if nlens > 1000:
        break

### Check that we can Read in the Spectra

In [ ]:
sglspec = read_spectra('tile000001_singleobj_spectra.fits')
lenspec = read_spectra('tile000001_lens_spectra.fits')
bkgspec = read_spectra('tile000001_bkgd_spectra.fits')
simspec = read_spectra('tile000001_simlens_spectra.fits')

In [ ]:
print(sglspec.num_spectra())
print(simspec.num_spectra())

### Make Plots

In [ ]:
mpl.rc('figure', max_open_warning = 0)

In [ ]:
from scipy.ndimage import gaussian_filter1d

for j in range(lenspec.num_spectra()):
    
    fig, axes = plt.subplots(1,3, figsize=(16,4.5), sharex=True, sharey=True, tight_layout=True)

    for b in 'brz':
        ax = axes[0]
        smoothed = gaussian_filter1d(lenspec.flux[b][j], 5)
        ax.plot(lenspec.wave[b], smoothed, lw=1, alpha=0.8, label=b)
        smoothed = gaussian_filter1d(lenspec.extra[b]['MODEL'][j], 5)
        ax.plot(bkgspec.wave[b], smoothed, lw=1, color='k', ls='--')
        ax.set(title='Lens: $z={:.3f}$ ({})'.format(lenspec.extra_catalog[j]['Z'], lenspec.extra_catalog[j]['SPECTYPE']),
               xlabel=r'$\lambda_\mathrm{obs}$ [$\AA$]',
               ylabel=r'flux [erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$]')
        ax.grid(ls=':')

        ax = axes[1]
        smoothed = gaussian_filter1d(bkgspec.flux[b][j], 5)
        ax.plot(bkgspec.wave[b], smoothed, lw=1, alpha=0.8)
        smoothed = gaussian_filter1d(bkgspec.extra[b]['MODEL'][j], 5)
        ax.plot(bkgspec.wave[b], smoothed, lw=1, color='k', ls='--')
        ax.set(title='Bkg: $z={:.3f}$ ({})'.format(bkgspec.extra_catalog[j]['Z'], bkgspec.extra_catalog[j]['SPECTYPE']),
               xlabel=r'$\lambda_\mathrm{obs}$ [$\AA$]')
        ax.grid(ls=':')

        ax = axes[2]
        smoothed = gaussian_filter1d(simspec.flux[b][j], 5)
        ax.plot(simspec.wave[b], smoothed, lw=1, alpha=0.8)
        alpha = simspec.extra_catalog['ALPHA'][j]
        ax.set(title=r'Combined: ${:.2f}\mathcal{{M}}_1 + {:.2f}\mathcal{{M}}_2$'.format(alpha, 1-alpha),
               xlabel=r'$\lambda_\mathrm{obs}$ [$\AA$]')
        ax.grid(ls=':')
        
    ax = axes[0]
    ax.legend(fontsize=12, ncol=1, loc='best')
        
#     fig.savefig('figures/tile{:06d}_synthlens{:03d}.png'.format(tile, j), dpi=120)
    if j >= 1:
        break

In [ ]:
sglspec.extra = None
sglspec_coadd = coadd_cameras(sglspec)

In [ ]:
write_spectra('tile{:06d}_singleobj_coadd.fits'.format(tileid), sglspec_coadd)

In [ ]:
coadd_test = read_spectra('tile000001_singleobj_coadd.fits')